In [4]:
import gurobipy as gp
import time
import json

from gurobipy import GRB
import numpy as np
rng = np.random.default_rng()

In [56]:
def JSONimport(siteJSON, connectionJSON):
    
    with open(siteJSON, "r") as f:
        sites = json.load(f)

    with open(connectionJSON, "r") as f:
        connections = json.load(f)

    numFacilities = int(len(list(connections))**.5)
    facilityNames = []
    for conn in list(connections)[0:numFacilities+3]:
        facilityNames.append(connections[conn]['dst'])
    print(facilityNames)
    
    ATLASGrid = {}
    for i in range(len(sites)):
        siteName = list(sites)[i]
        ATLASGrid[i] = {}
        ATLASGrid[i]['siteName'] = siteName
        ATLASGrid[i]['sitePerformance'] = sites[siteName]['GFLOPS']
        storageName = list(sites[siteName]['RSE'])[0]

        exceptions = {'BNL_PROD_INTEL':'BNL-ATLAS', 'BNL_VHIMEM':'BNL-ATLAS','SLAC':'BNL-ATLAS'}
        for facility in facilityNames:
            if facility in storageName:
                ATLASGrid[i]['facility'] = facility
                break
            elif facility in siteName or siteName in facility:
                ATLASGrid[i]['facility'] = facility
                break
        try:
            _ = ATLASGrid[i]['facility']
        except:
            ATLASGrid[i]['facility'] = exceptions[siteName]
            
    return ATLASGrid, facilityNames

In [57]:
JSONimport('ATLASGridDev/site_info.json','ATLASGridDev/site_conn_info.json')

['AGLT2', 'ARNES', 'AYDIN', 'AZ-IFAN', 'BEIJING-LCG2', 'BNL-ATLAS', 'BNL-ATLAS-OPP', 'BNLHPC', 'BOINC', 'CA-SFU-T2', 'CA-VICTORIA-WESTGRID-T2', 'CA-WATERLOO-T2', 'CERN-EXTENSION', 'CERN-P1', 'CERN-PROD', 'CSCS-LCG2', 'CYFRONET-LCG2', 'DESY-HH', 'DESY-ZN', 'EELA-UNLP', 'EELA-UTFSM', 'FMPhI-UNIBA', 'FZK-LCG2', 'GOOGLE', 'GOOGLE_COMPUTE_ENGINE', 'GR-12-TEIKAV', 'GRIF', 'GRIF-IRFU', 'GRIF-LAL', 'GRIF-LPNHE', 'GoeGrid', 'HELIX_NEBULA', 'HEPHY-UIBK', 'HK-LCG2', 'IAAS', 'IEPSAS-Kosice', 'IFIC-LCG2', 'IL-TAU-HEP', 'IN2P3-CC', 'IN2P3-CC-T3', 'IN2P3-CPPM', 'IN2P3-LAPP', 'IN2P3-LPC', 'INFN-BOLOGNA-T3', 'INFN-COSENZA', 'INFN-FRASCATI', 'INFN-GENOVA', 'INFN-LECCE', 'INFN-MILANO-ATLASC', 'INFN-NAPOLI-ATLAS', 'INFN-ROMA1', 'INFN-ROMA2', 'INFN-ROMA3', 'INFN-T1', 'INFN-TRIESTE', 'ITEP', 'JINR-LCG2', 'LBNL_DSD_ITB', 'LRZ-LMU', 'MPPMU', 'MWT2', 'Microsoft-Azure', 'NCG-INGRID-PT', 'NDGF-T1', 'NERSC-PDSF', 'NET2', 'NIKHEF', 'NO-NORGRID-T2', 'Nevis', 'OUHEP_OSG', 'OU_OSCER_ATLAS', 'PSNC', 'RAL-LCG2', 'RAL-L

({0: {'siteName': 'AGLT2', 'sitePerformance': 28407840, 'facility': 'AGLT2'},
  1: {'siteName': 'AGLT2_MERGE',
   'sitePerformance': 4679040,
   'facility': 'AGLT2'},
  2: {'siteName': 'AGLT2_VHIMEM',
   'sitePerformance': 330720,
   'facility': 'AGLT2'},
  3: {'siteName': 'ANALY_AGLT2_VP',
   'sitePerformance': 8948640,
   'facility': 'AGLT2'},
  4: {'siteName': 'ANALY_ARNES_DIRECT',
   'sitePerformance': 20022400,
   'facility': 'ARNES'},
  5: {'siteName': 'ANALY_BNL_VP',
   'sitePerformance': 1412640,
   'facility': 'BNL-ATLAS'},
  6: {'siteName': 'ANALY_LRZ_VP',
   'sitePerformance': 3535200,
   'facility': 'LRZ-LMU'},
  7: {'siteName': 'ANALY_SWT2_CPB_VP',
   'sitePerformance': 859200,
   'facility': 'SWT2_CPB'},
  8: {'siteName': 'ANALY_SiGNET_DIRECT',
   'sitePerformance': 10517760,
   'facility': 'NDGF-T1'},
  9: {'siteName': 'ARC-TEST',
   'sitePerformance': 232960,
   'facility': 'NDGF-T1'},
  10: {'siteName': 'ARC-TEST_CONDOR',
   'sitePerformance': 2720,
   'facility': 'NDG

In [25]:
def updateX(oldX, oldY, oldZ, J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance,timeLimit):
    model = gp.Model('MILP')
    currentTime = time.time()
    
    #binary variable
    X = model.addVars(range(J), range(C), vtype=GRB.BINARY, name="X")
    updatedTime = time.time()
    print(f'X variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    
    #continuous variables
    u = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="u")
    updatedTime = time.time()
    print(f'u variables created.This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    v = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="v")
    updatedTime = time.time()
    print(f'v variables created.This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    e = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="e")
    updatedTime = time.time()
    print(f'e variables created.This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    t = model.addVars(range(D), vtype=GRB.CONTINUOUS, name="t")
    updatedTime = time.time()
    print(f't variables created.This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #objective
    m = model.addVar(vtype=GRB.CONTINUOUS, name='makespan')
    updatedTime = time.time()
    print(f'm variable created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #initialize X
    for j in range(J):
        for c in range(C):
            X[j,c].Start = oldX[j][c]
    updatedTime = time.time()
    print(f'X initialized. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #12
    for j in range(J):
        model.addConstr(m >= v[j] + e[j])
    updatedTime = time.time()
    print(f"Equation 12 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #13
    for j in range(J):
        model.addConstr(gp.quicksum(X[j,c] for c in range(C)) == 1)
    updatedTime = time.time()
    print(f"Equation 13 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #14
    for j in range(J):
        model.addConstr(e[j] == gp.quicksum(X[j,c] * Gamma * gp.quicksum(sd/performance[c] for sd in filesPerJ[j]) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 14 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #omit 15-16 because they are contraints on Y but we update X

    #17
    for j in range(J):
        for i in range(J):
            if i != j:
                for c in range(C):
                    model.addLConstr(u[j] >= A * (oldY[i][j] * (X[i,c]+X[j,c]-1) - 1) + v[i] + e[i])
    updatedTime = time.time()
    print(f"Equation 17 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #omit 18 because it is a constraint on Z but we update X

    #19
    for d in range(D):
        locationD = I[d]
        remoteServer = [i for i in range(R) if locationD[i] == 1][0]
        model.addConstr(t[d] == gp.quicksum((sizePerFile[d]/Bremote[remoteServer][l]) * oldZ[d][l] for l in range(L)))
    updatedTime = time.time()
    print(f"Equation 19 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #20
    for j in range(J):
        for file in dPerJob[j]:
            model.addLConstr(v[j] >= t[d] + gp.quicksum((sizePerFile[file]/Blocal[l][c]) * X[j,c] * oldZ[file][l] for l in range(L) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 20 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #21
    for j in range(J):
        for file in dPerJob[j]:
            model.addLConstr(v[j] >= u[j] + gp.quicksum((sizePerFile[file]/Blocal[l][c]) * X[j,c] * oldZ[file][l] for l in range(L) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 21 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    model.setObjective(m, GRB.MINIMIZE)
    model.setParam('TimeLimit',timeLimit)
    model.update()
    print('Model Begins')
    model.optimize()
    newX = [[X[j,c].X for c in range(C)] for j in range(J)]

    return newX

In [ ]:
def updateYZ(oldX, oldY, oldZ, J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance,timeLimit):
    model = gp.Model('MILP')
    currentTime = time.time()
    
    #binary variable
    Y = model.addVars(range(J), range(J), vtype=GRB.BINARY, name="Y")
    updatedTime = time.time()
    print(f'Y variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    Z = model.addVars(range(D), range(L), vtype=GRB.BINARY, name="Z")
    updatedTime = time.time()
    print(f'Z variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #continuous variables
    u = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="u")
    updatedTime = time.time()
    print(f'u variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    v = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="v")
    updatedTime = time.time()
    print(f'v variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    e = model.addVars(range(J), vtype=GRB.CONTINUOUS, name="e")
    updatedTime = time.time()
    print(f'e variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    t = model.addVars(range(D), vtype=GRB.CONTINUOUS, name="t")
    updatedTime = time.time()
    print(f't variables created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #objective
    m = model.addVar(vtype=GRB.CONTINUOUS, name='makespan')
    updatedTime = time.time()
    print(f'm variable created. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime
    
    #initialize Y and Z
    for i in range(J):
        for j in range(J):
            Y[i,j].Start = oldY[i][j]
    updatedTime = time.time()
    print(f'Y initialized. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    for d in range(D):
        for l in range(L):
            Z[d,l].Start = oldZ[d][l]
    updatedTime = time.time()
    print(f'Z initialized. This took {updatedTime - currentTime} seconds')
    currentTime = updatedTime

    #12
    for j in range(J):
        model.addConstr(m >= v[j] + e[j])
    updatedTime = time.time()
    print(f"Equation 12 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #omit 13 because it is a constraint in X but we update Y and Z

    #14
    for j in range(J):
        model.addConstr(e[j] == gp.quicksum(oldX[j][c] * Gamma * gp.quicksum(sd/performance[c] for sd in filesPerJ[j]) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 14 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #15 and 16
    for i in range(J):
        for j in range(J):
            if i != j:
                model.addConstr(Y[i,j] + Y[j,i] == 1)
            else:
                model.addConstr(Y[i,j] == 0)
    updatedTime = time.time()
    print(f"Equation 15 and 16 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime
    
    #17
    for j in range(J):
        for i in range(J):
            if i != j:
                for c in range(C):
                    model.addLConstr(u[j] >= A * (Y[i,j] * (oldX[i][c]+oldX[j][c]-1) - 1) + v[i] + e[i])
    updatedTime = time.time()
    print(f"Equation 17 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #18
    for d in range(D):
        model.addConstr(gp.quicksum(Z[d,l] for l in range(L)) == 1)
    updatedTime = time.time()
    print(f"Equation 18 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #19
    for d in range(D):
        locationD = I[d]
        remoteServer = [i for i in range(R) if locationD[i] == 1][0]
        model.addConstr(t[d] == gp.quicksum((sizePerFile[d]/Bremote[remoteServer][l]) * Z[d,l] for l in range(L)))
    updatedTime = time.time()
    print(f"Equation 19 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime
    
    #20
    for j in range(J):
        for file in dPerJob[j]:
            model.addLConstr(v[j] >= t[d] + gp.quicksum((sizePerFile[file]/Blocal[l][c]) * oldX[j][c] * Z[file,l] for l in range(L) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 20 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime

    #21
    for j in range(J):
        for file in dPerJob[j]:
            model.addLConstr(v[j] >= u[j] + gp.quicksum((sizePerFile[file]/Blocal[l][c]) * oldX[j][c] * Z[file,l] for l in range(L) for c in range(C)))
    updatedTime = time.time()
    print(f"Equation 21 is added. This took {updatedTime - currentTime} seconds")
    currentTime = updatedTime
    
    model.setObjective(m, GRB.MINIMIZE)
    model.setParam('TimeLimit',timeLimit)
    model.update()
    print('Model Begins')
    model.optimize()
    newY = [[Y[i,j].X for j in range(J)] for i in range(J)]
    newZ = [[Z[d,l].X for l in range(L)] for d in range(D)]

    return newY,newZ


In [ ]:
def XYZtoM(X,Y,Z,J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance):
    #Assign each job to a node by using solution X
    jobsPerNode = [[] for c in range(C)]
    dataPerNode = [[] for c in range(C)]

    for j in range(J):
        for c in range(C):
            if round(X[j][c]) == 1:
                jobsPerNode[c].append(j)
                dataPerNode[c].append(filesPerJ[j])

    #Order jobs with using solution Y. If Y[x][y] == 1, x comes before y
    jobsPerNodeOrdered = [[] for c in range(C)]
    dataPerNodeOrdered = [[] for c in range(C)]

    for n in range(C):
        node = jobsPerNode[n]
        data = dataPerNode[n]
        if len(node) == 0:
            continue
        for j in range(len(node)):
            job = node[j]
            datum = data[j]
            if j == 0:
                jobsPerNodeOrdered[n].append(job)
                dataPerNodeOrdered[n].append(datum)
                continue
            for k in range(j):
                if round(Y[j][k]) == 1:
                    jobsPerNodeOrdered[n].insert(k, job)
                    dataPerNodeOrdered[n].insert(k, datum)
                    break
                jobsPerNodeOrdered[n].append(job)
                dataPerNodeOrdered[n].append(datum)

    #Calculate transfer and execution time with using bandwidths and performance parameters
    transferTimes = [[] for c in range(C)]
    executionTimes = [[] for c in range(C)]

    for node in range(C):
        for job in range(len(jobsPerNodeOrdered[node])):
            jobID = jobsPerNodeOrdered[node][job]
            dataSize = 0
            transferTime = 0
            for data in dataPerNodeOrdered[node][job]:
                dataSize += data
                d = [i for i in dPerJob[jobID] if data == sizePerFile[i]][0]
                remoteLocation = [remote for remote in range(len(I[d])) if I[d][remote] == 1][0]
                localLocation = [local for local in range(len(Z[d])) if Z[d][local] == 1][0]
                remote2local = dataSize / Bremote[remoteLocation][localLocation]
                local2node = dataSize / Blocal[localLocation][node]
                if remote2local+local2node > transferTime:
                    transferTime = remote2local+local2node
            executionTime = dataSize / performance[node]
            transferTimes[node].append(transferTime)
            executionTimes[node].append(executionTime)

    #find makespan for each node, the largest is the total makespan. (We can keep older makespan to use in real time application of the algorithm)
    m = 0
    for node in range(C):
        if jobsPerNode[node] == []:
            continue
        nodeSpan = 0
        nodeSpan += transferTimes[node][0]
        transferedTime = transferTimes[node][0]
        for job in range(1,len(jobsPerNode[node])):
            transferedTime += transferTimes[node][job]
            nodeSpan = max(nodeSpan + executionTimes[node][job-1], transferedTime)
        nodeSpan += executionTimes[node][-1]
        if nodeSpan > m:
            m = nodeSpan

    return m

In [ ]:
def randomXYZ(J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,I):

    X = []
    for j in range(J):
        cRange = [0 for c in range(C)]
        cRange[rng.integers(0,C)] = 1
        X.append(cRange)

    Y = [[0 for j in range(J)] for jj in range(J)]
    for j in range(1,J):
        for jj in range(j):
            temp = int(rng.integers(0,2))
            Y[j][jj] = temp
            Y[jj][j] = 1 - temp

    Z = []
    for d in range(D):
        lRange = [0 for l in range(L)]
        lRange[rng.integers(0,L)] = 1
        Z.append(lRange)

    return X,Y,Z

In [ ]:
Gurobis = []
Randoms = []
for run in range(10):
  J = 200 #number of jobs
  C = 148 #number of computational nodes (SITE ENTRIES FROM ATLAS CRIC)
  D = 0 #number of data files, only initiation, value is set in definition of job files
  Gamma = 20 #computation coefficient
  R = 16 #number of remote storage nodes (TAPE ENTRIES FROM ATLAS CRIC)
  L = 114 #number of local storage nodes (DISK ENTRIES FROM ATLAS CRIC)
  timeLimit = 50
  A = 1e6
  print('==================================================')
  print(f'SIMULATION RUN {run}')
  print('==================================================')
  simStart = time.time()
  filesPerJ = []
  sizePerFile = []
  dataToJob = []
  dBeginningPerJob = []
  dPerJob = []
  for job in range(J):
    dJob = []
    dBeginningPerJob.append(D)
    numOfFiles = rng.integers(5,11)
    files = []
    for file in range(numOfFiles):
      fileSize = rng.normal(1e5,1e3)
      files.append(fileSize)
      sizePerFile.append(fileSize)
      dataToJob.append(job)
      dJob.append(D)
      D += 1
    dPerJob.append(dJob)
    filesPerJ.append(files)
  print('Files created for each job.')

  I = []
  for file in range(D):
    remotePlacement = [0 for i in range(R)]
    remotePlacement[rng.integers(0,R)] = 1
    I.append(remotePlacement)
  print('Files placed to remote storages.')

  Bremote = []
  for remote in range(R):
    remoteLocalBW = []
    for local in range(L):
      remoteLocalBW.append(rng.integers(700,1300) * 1e3 * 8) 
    Bremote.append(remoteLocalBW)
  print('Bandwidth is set between remote and local storage.')

  Blocal = []
  for local in range(L):
    localNodeBW = []
    for node in range(C):
      localNodeBW.append(rng.integers(7000,13000) * 1e3 * 8) 
    Blocal.append(localNodeBW)
  print('Bandwidth is set between local storage and computing sites.')

  performance = []
  for cpu in range(C):
    cores = 2 ** rng.integers(4,7)
    performance.append(500 * cores)
  print('CPU Performances are set.')

  tempX = []
  for j in range(J):
    jobRange = []
    for c in range(C):
      jobRange.append(rng.integers(0,2))
    tempX.append(jobRange)
  print('Temporary X created.')

  tempY = []
  for j in range(J):
    jobRange = []
    for jj in range(J):
      jobRange.append(rng.integers(0,2))
    tempY.append(jobRange)
  print('Temporary Y created.')

  tempZ = []
  for d in range(D):
    dataRange = []
    for l in range(L):
      dataRange.append(rng.integers(0,2))
    tempZ.append(dataRange)
  print('Temporary Z created.')

  for T in range(2):
    print('--------------------------------------------------')
    print(f'CYCLE {T}')
    print('--------------------------------------------------')
    print('--------------------UPDATE X--------------------')
    newX = updateX(tempX, tempY, tempZ, J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance,timeLimit)
    if T == 0:
        tempX = newX
    print('--------------------UPDATE YZ--------------------')
    newY, newZ = updateYZ(tempX, tempY, tempZ, J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance,timeLimit)
    tempX, tempY, tempZ = newX, newY, newZ

  simEnd = time.time()
  print(f'Simulation run {run} took {simEnd - simStart} seconds.')

  randomX, randomY, randomZ = randomXYZ(J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,I)

  mGurobi = XYZtoM(tempX,tempY,tempZ,J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance)
  mRandom = XYZtoM(randomX,randomY,randomZ,J,C,D,Gamma,R,L,A,filesPerJ,sizePerFile,dPerJob,I,Bremote,Blocal,performance)
  Gurobis.append(mGurobi)
  Randoms.append(mRandom)

In [ ]:
print(np.average(Gurobis))
print(np.average(Randoms))
print(f'Reduction:{(np.average(Gurobis) - np.average(Randoms))/np.average(Randoms) * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Gurobis)
plt.plot(Randoms)

In [ ]:
print(Gurobis)

In [ ]:
print(Randoms)